In [ ]:
import pandas as pd
import numpy as np
import re
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, EarlyStoppingCallback, Trainer, TrainingArguments
import evaluate
import os
import torch

print(torch.cuda.is_available())

MAX_LENGTH = 152
NUM_LABELS = 3
EPOCHS = 3
BATCH_SIZE = 32
BASE_MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"
FINE_TUNED_MODEL_DIR = "fine_tuned_sentiment_model"

def clean_text(text):
    """Clean and preprocess text data"""
    if pd.isna(text) or text is None:
        return ""
    
    text = str(text)
    
    # Remove URLs, email addresses, whitespace
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

def validate_and_clean_data(df):
    """Comprehensive data validation and cleaning"""
    print(f"Initial dataset size: {len(df)} rows")
    
    # Check required columns
    required_cols = ["review", "label"]
    for col in required_cols:
        if col not in df.columns:
            raise KeyError(f"Required column '{col}' not found in CSV!")
    
    # Handle missing review text
    print(f"Missing review texts: {df['review'].isna().sum()}")
    df = df.dropna(subset=['review'])
    
    # Clean text data
    print("Cleaning text data...")
    df['review'] = df['review'].apply(clean_text)
    
    # Remove empty texts after cleaning
    df = df[df['review'].str.len() > 0]
    print(f"After removing empty texts: {len(df)} rows")
    
    # Remove very short texts (less than 3 characters)
    df = df[df['review'].str.len() >= 3]
    print(f"After removing very short texts: {len(df)} rows")
    
    # Remove very long texts (keep within reasonable limits)
    df = df[df['review'].str.len() <= 1000]
    print(f"After removing very long texts: {len(df)} rows")
    
    # Label validation and cleaning
    print(f"Missing labels: {df['label'].isna().sum()}")
    
    # Convert labels to numeric, invalid ones become NaN
    df['label'] = pd.to_numeric(df['label'], errors='coerce')
    
    # Remove rows with invalid labels
    df = df.dropna(subset=['label'])
    print(f"After removing invalid labels: {len(df)} rows")
    
    # Ensure labels are within valid range (0, 1, 2 for sentiment)
    valid_labels = df['label'].isin([0, 1, 2])
    invalid_count = (~valid_labels).sum()
    if invalid_count > 0:
        print(f"Removing {invalid_count} rows with invalid label values")
        df = df[valid_labels]
    
    # Convert to int after validation
    df['label'] = df['label'].astype(int)
    
    # Remove duplicates based on review text
    initial_size = len(df)
    df = df.drop_duplicates(subset=['review'], keep='first')
    duplicates_removed = initial_size - len(df)
    print(f"Removed {duplicates_removed} duplicate reviews")
    
    # Show label distribution
    print(f"\nLabel distribution:")
    print(df['label'].value_counts().sort_index())
    
    print(f"\nFinal cleaned dataset size: {len(df)} rows")
    
    # Reset index after all the filtering
    df = df.reset_index(drop=True)
    
    return df

def tokenize(batch):
    texts = [str(x) if x is not None else "" for x in batch["review"]]
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH
    )

if __name__ == "__main__":
    # Load and clean data with robust error handling
    try:
        df = pd.read_csv(
            "training_data.csv",
            quotechar='"',
            escapechar='\\',
            engine="python",
            on_bad_lines="skip"
        )
    except Exception as e:
        print(f"Error reading CSV: {e}")
        print("Trying with basic pandas read_csv...")
        df = pd.read_csv("training_data.csv")
    
    # Apply comprehensive data cleaning
    df = validate_and_clean_data(df)
    
    # Check if we have enough data after cleaning
    if len(df) < 10:
        raise ValueError(f"Insufficient data after cleaning: only {len(df)} rows remaining")
    
    dataset = Dataset.from_pandas(df)

    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

    # Use fewer processes on Windows to avoid hang
    dataset = dataset.map(tokenize, batched=True, num_proc=1)
    dataset = dataset.rename_column("label", "labels")
    dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    dataset = dataset.train_test_split(test_size=0.2)
    train_dataset = dataset["train"]
    eval_dataset = dataset["test"]

    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")

    def compute_metrics(pred):
        logits = pred.predictions
        labels = pred.label_ids
        preds = np.argmax(logits, axis=-1)
        acc = accuracy.compute(predictions=preds, references=labels)
        f1_score = f1.compute(predictions=preds, references=labels, average="weighted")
        return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}

    model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_NAME, num_labels=NUM_LABELS)

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",  # evaluate less often
        eval_steps=500,
        save_strategy="steps",
        save_steps=500,
        save_total_limit=2,
        learning_rate=2e-5,
        per_device_train_batch_size=32,  # larger batch if possible
        per_device_eval_batch_size=32,
        num_train_epochs=EPOCHS,
        weight_decay=0.01,
        warmup_steps=50,
        fp16=True,  # mixed precision
        logging_dir="./logs",
        logging_steps=500,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True
    )
    model.gradient_checkpointing_enable()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    print("\nStarting fine-tuning...")
    trainer.train()

    trainer.save_model(FINE_TUNED_MODEL_DIR)
    tokenizer.save_pretrained(FINE_TUNED_MODEL_DIR)
    print(f"Fine-tuned model saved to {FINE_TUNED_MODEL_DIR}")

False
Initial dataset size: 7472 rows
Missing review texts: 0
Cleaning text data...
After removing empty texts: 7472 rows
After removing very short texts: 7472 rows
After removing very long texts: 7435 rows
Missing labels: 0
After removing invalid labels: 7435 rows
Removed 184 duplicate reviews

Label distribution:
label
0    1095
1    1892
2    4264
Name: count, dtype: int64

Final cleaned dataset size: 7251 rows


C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/7251 [00:00<?, ? examples/s]

C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(



Starting fine-tuning...


...

  0%|          | 0/546 [00:00<?, ?it/s]

C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.3399, 'grad_norm': 6.6044697761535645, 'learning_rate': 1.8548387096774196e-06, 'epoch': 2.75}


  0%|          | 0/46 [00:00<?, ?it/s]

{'eval_loss': 0.4641847610473633, 'eval_accuracy': 0.8456237077877325, 'eval_f1': 0.8448316496857904, 'eval_runtime': 572.091, 'eval_samples_per_second': 2.536, 'eval_steps_per_second': 0.08, 'epoch': 2.75}
{'train_runtime': 60707.2386, 'train_samples_per_second': 0.287, 'train_steps_per_second': 0.009, 'train_loss': 0.3262126122638856, 'epoch': 3.0}
Fine-tuned model saved to fine_tuned_sentiment_model
